Lo primero se debe imporatr las librerias que vamosa utlizar para cargar losda tos a la abse de datos, que ene sta caso son pandas para el manejo de losd atos. 

In [ ]:
import pandas as pd
import psycopg2
from src.database.db_conection import connect_db


In [ ]:
data = pd.read_csv('../data/candidates.csv')

In [ ]:
# Cargar el archivo CSV
csv_path = "data/candidates.csv"
df = pd.read_csv(csv_path)

# Conectar a la base de datos
conn = connect_db()
cursor = conn.cursor()

# Crear la tabla si no existe
cursor.execute("""
    CREATE TABLE IF NOT EXISTS candidates (
        id SERIAL PRIMARY KEY,
        first_name VARCHAR(100),
        last_name VARCHAR(100),
        email VARCHAR(100) UNIQUE,
        country VARCHAR(50),
        application_date DATE,
        yoe INT,
        seniority VARCHAR(50),
        technology VARCHAR(100),
        code_challenge_score FLOAT,
        technical_interview FLOAT
    );
""")
conn.commit()

# Insertar datos en la tabla
for _, row in df.iterrows():
    cursor.execute("""
        INSERT INTO candidates (first_name, last_name, email, country, application_date, yoe, seniority, technology, code_challenge_score, technical_interview)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (email) DO NOTHING;
    """, (
        row["First Name"], row["Last Name"], row["Email"], row["Country"],
        row["Application Date"], row["Yoe (years of experience)"],
        row["Seniority"], row["Technology"],
        row["Code Challenge Score"], row["Technical Interview"]
    ))

conn.commit()
cursor.close()
conn.close()

print("Datos cargados correctamente en la base de datos.")